## syft==0.2.4 教程

## 1 张量通信

VirtualWorker可以简单理解为一个远程机器。x和y是两个张量，x_ptr和y_ptr是x和y的指针。

这里有一个方法：.send()。它的作用是把张量发送到远程机器，在发送之后，本机依然保留了它的操作权，就是通过它返回的指针进行操作。

下面一句很关键：z_ptr = x_ptr + x_ptr。
在原教程中写的是z = x_ptr + x_ptr。我觉得命名有误，因为此时，z并不是一个实际的张量，而是一个指针。

这里的x_ptr和y_ptr都不是实际数据，但却可以执行加法操作，事实上这里是发送了一个操作到远程机器，让远程机器在数据上执行加法，而其产生的结果也是一个指针，指向的是保留在远程机器上的结果，通过get()获取其真实数据，并且在获取后，远程的bob将失去这个数据，这就是将数据所有权归还给了本地——数据所有权是传递的。

In [2]:
import numpy as np 
import syft as sy # Run this cell to see if things work
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [3]:
# 基本张量运算
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


In [32]:
# 张量指针的运算
bob = sy.VirtualWorker(hook, id="bob")
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])
x_ptr = x.send(bob)
y_ptr = y.send(bob)
z_ptr = x_ptr + x_ptr
z = z_ptr.get()

## 2 张量指针

张量（tensor）是数据科学、深度学习中的一个基本概念，用过pytorch、tensorflow的会对它有更深的认识。

这里不详细阐述张量的概念，可以去之前学tensorflow的文章中看看。这里只谈用PySyft是如何解决安全隐私问题的：

张量通常包含数据，数据可能包含隐私信息，很多时候计算的任务不能独立完成，需要借助第三方，在这个过程中，必须保留数据持有者对数据的操作权，PySyft就是基于这个思想，提出了张量指针（PointerTensor）的概念。

指针我们都知道，学过计算机语言的都能说出“地址”等概念。但这里的张量指针并不只是变量地址这么简单的东西。

In [38]:
bob = sy.VirtualWorker(hook, id="tom")

In [39]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)
z = x+y
print(z)

(Wrapper)>[PointerTensor | me:41193493464 -> tom:49833880055]


In [41]:
z.get()

tensor([2, 3, 4, 5, 6])

In [43]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)
z = (x + y).sum()
z.backward()
x = x.get()
print(x)
print(x.grad)

tensor([1., 2., 3., 4., 5.], requires_grad=True)


## 3 worker工作机
1. 工作机（Worker），它表示一台拥有计算资源和数据资源的实体。之前的VirtualWorker就是对这样一个实体的模拟，用于演示与远程机器的通信。

2. 本地工作机的计算资源和数据资源就是原生的torch操作和张量。

3. 它在调用hook的时候会自动创建。

4. 工作机的一个基本原则是，它只能对自己的机器上的数据进行计算。

5. 事实上，除了数据只能使用工作机所有，“计算”也是一样，只是在上面进行加操作的每一步，事实上都是把每一个计算操作发送到了远程工作机上。下面的“计划”会进一步说明。

In [ ]:
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
# alice和bob是远程的工作机
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])
# x y都是本地的数据
z = x + y # z 也是本地的
# 将x发送到alice、y发送到bob
x_ptr = x.send(alice)
y_ptr = y.send(bob)
# 这一句不能执行，因为x_ptr是alice的数据，y_ptr是bob的数据
z = x_ptr + y
# 可以执行，x_ptr和y_ptr此时都在bob上
x_ptr = x.send(bob)
z = x_ptr+y_ptr